## Testing NC backboning over time (across 20 years)

Playing around with the module in util (had to write one from scratch as there is no noise corrected function in the backbone module) and different alpha values

In [20]:
import sys
import os
import pandas as pd
import pickle

with open('../data/cartel_proj_by_year.pkl', 'rb') as f:
    graphs_by_year = pickle.load(f)

# add parent directory to path so we can import from util
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# import module from util
from util.nc_backbone import nc_backbone

alpha_value = 0.999

for year in range(1990, 2011):
    adj = pd.read_csv(f'../data/cartel_adj_mat/{year}.csv', index_col=0)
    print(f'Year: {year}, Backbone edges: {len(nc_backbone(adj, alpha=alpha_value))}, Original edges: {graphs_by_year[year].number_of_edges()}')


Year: 1990, Backbone edges: 0, Original edges: 0
Year: 1991, Backbone edges: 0, Original edges: 0
Year: 1992, Backbone edges: 0, Original edges: 1
Year: 1993, Backbone edges: 2, Original edges: 2
Year: 1994, Backbone edges: 2, Original edges: 2
Year: 1995, Backbone edges: 2, Original edges: 2
Year: 1996, Backbone edges: 5, Original edges: 5
Year: 1997, Backbone edges: 2, Original edges: 2
Year: 1998, Backbone edges: 4, Original edges: 4
Year: 1999, Backbone edges: 2, Original edges: 2
Year: 2000, Backbone edges: 4, Original edges: 4
Year: 2001, Backbone edges: 13, Original edges: 13
Year: 2002, Backbone edges: 5, Original edges: 5
Year: 2003, Backbone edges: 13, Original edges: 13
Year: 2004, Backbone edges: 18, Original edges: 19
Year: 2005, Backbone edges: 21, Original edges: 25
Year: 2006, Backbone edges: 32, Original edges: 37
Year: 2007, Backbone edges: 31, Original edges: 41
Year: 2008, Backbone edges: 23, Original edges: 45
Year: 2009, Backbone edges: 20, Original edges: 45
Year

Smaller (and more reasonable) alpha values lead to very sparse backbones :(  
Filtering out the edge **(Golfo, Zetas)** before adding it to the sum of total weights not only does not help, but also returns less edges for the same alpha values without previous filtering. It is probably related to the fact that the prob of success increases, and the expected weight is way larger compared to the actual one.  
Interestingly, for the same alpha value, the networks with the most edges retained are not the most densely connected ones but networks from 2006 and 2007 (they are not cliques, in contrast with networks from 2008-2010)

TBC: Visualizing the graph evolution over the years after using heuristics to pick an alpha value and therefore have the final graphs we will work with